<a href="https://colab.research.google.com/github/tejjusbhat/SaaS-Customer-Churn-Prediction/blob/main/Churn_Prediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import pandas as pd

In [2]:
df = pd.read_csv("saas_churn_clean.csv")

In [3]:
df.head()

,age,gender,region_category,joined_through_referral,preferred_offer_types,medium_of_operation,internet_option,days_since_last_login,avg_session_duration,avg_transaction_value,...,session_minutes_90d,days_since_active,api_calls_90d_log,api_usage_bucket,recency_bucket,logins_per_active_day,minutes_per_login,usage_intensity,tenure_days,last_visit_hour
0,18,F,Village,No,Gift Vouchers/Coupons,?,Wi-Fi,17.0,300.63,53005.25,...,1027.11,2.0,9.081825,2.0,0.0,1.352941,21.853404,32.566667,2918.0,16.0
1,32,F,City,?,Gift Vouchers/Coupons,Desktop,Mobile_Data,16.0,306.34,12838.38,...,862.65,1.0,9.060215,2.0,0.0,1.193548,22.701316,47.805556,2907.0,12.0
2,44,F,Town,Yes,Gift Vouchers/Coupons,Desktop,Wi-Fi,14.0,516.16,21027.00,...,411.37,16.0,5.590987,0.0,1.0,0.944444,22.853889,0.174510,3197.0,22.0
3,37,M,City,Yes,Gift Vouchers/Coupons,Desktop,Mobile_Data,11.0,53.27,25239.56,...,215.77,20.0,5.429346,0.0,1.0,0.888889,23.974444,0.120106,3210.0,15.0
4,31,F,City,No,Credit/Debit Card Offers,Smartphone,Mobile_Data,20.0,113.13,24483.66,...,231.94,2.0,5.686975,0.0,0.0,1.083333,16.567143,1.088889,2892.0,15.0
